## Populate a Property Graph

This notebook reports the main steps to download CSV files, process them and create a property graph.

To measure execution time in Jupyter notebooks: <code>pip install ipython-autotime</code>

To connect to Neo4j: <code>pip install neo4j</code>



In [2]:
# Required libraries
import pandas as pd
import os
import gc
from pathlib import Path
from datetime import datetime

### Connection to Neo4j

In [3]:
# Neo4J params class
class Neo4jParams:
  def __init__(self, user, psw,dbname,db_psw,uri):
    self.user = user
    self.psw = psw
    self.dbname = dbname
    self.dbpsw = dbpsw
    self.uri = uri

In [4]:
#DB parameters

user="neo4j"
#psw="password"
psw="AirTraffic_DB"
dbname="neo4j"
dbpsw="AirTraffic_DB"
uri = "bolt://localhost:7687"

params = Neo4jParams(user,psw,dbname,dbpsw,uri)


In [5]:
from neo4j import GraphDatabase

# test class

class Driver:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def print_greeting(self, message):
        with self.driver.session() as session:
            greeting = session.write_transaction(self._create_and_return_greeting, message)
            print(greeting)

    @staticmethod
    def _create_and_return_greeting(tx, message):
        result = tx.run("CREATE (a:Greeting) "
                        "SET a.message = $message "
                        "RETURN a.message + ', from node ' + id(a)", message=message)
        return result.single()[0]


if __name__ == "__main__":
    greeter = Driver("bolt://localhost:7687", "neo4j", "AirTraffic_DB")
    greeter.print_greeting("hello, world")
    greeter.close()

hello, world, from node 186954


## Data ingestion

In [6]:
# Parameters and URLS
path = str(Path(os.path.abspath(os.getcwd())).parent.absolute())

flightsPath = path + '/data/flights/'
flightsFIRPath = path + '/data/flights_FIR/'

sampledFlightsURL = flightsPath + 'Flights_marchSample.csv'
sampledFIRsURL = flightsFIRPath + 'Flight_FIRs_Actual_marchSample.csv'
airportsURL = path + '/data/airport-codes.csv'
aircraftURL = path + '/data/aircraft.csv'
aircraftTypesURL = path + '/data/doc8643AircraftTypes.csv'
airlineURL = path + '/data/airline.csv'
manufacturersURL = path + '/data/doc8643Manufacturers.csv'

# Country codes
countriesURL = path + '/data/wikipedia-iso-country-codes.csv'

# Cities codes
citiesURL = path + '/data/world-cities_csv.csv'

# FIR codes from wikipedia
firWikiUrl = path + '/data/fir_wiki.csv'

# FIR coordinates
firCoordURL = flightsFIRPath + '/FIR_1904.csv'

### Constraints
In this section we create all the constraints we need in the database.

In [6]:
# Connect to the DB
driver = GraphDatabase.driver(params.uri, auth=(params.dbname, params.dbpsw))
# Create a session
session = driver.session()

In [7]:
# Unique Node Constraints

session.run("CREATE CONSTRAINT flight_id IF NOT EXISTS ON (f:Flight) ASSERT f.id IS UNIQUE")
session.run("CREATE CONSTRAINT airport_id IF NOT EXISTS ON (a:Airport) ASSERT a.icao IS UNIQUE")
session.run("CREATE CONSTRAINT city_id IF NOT EXISTS ON (c:City) ASSERT c.id IS UNIQUE")
session.run("CREATE CONSTRAINT country_id IF NOT EXISTS ON (c:Country) ASSERT c.isocode IS UNIQUE")
session.run("CREATE CONSTRAINT airspace_id IF NOT EXISTS ON (f:Airspace) ASSERT (f.icao, f.type) IS NODE KEY")
session.run("CREATE CONSTRAINT airline_id IF NOT EXISTS ON (a:Airline) ASSERT a.icao IS UNIQUE")
session.run("CREATE CONSTRAINT aircraft_id IF NOT EXISTS ON (a:AircraftModel) ASSERT a.icao IS UNIQUE")
session.run("CREATE CONSTRAINT manufacturer_id IF NOT EXISTS ON (m:Manufacturer) ASSERT m.code IS UNIQUE")

In [8]:
driver.close()

### Countries

In [9]:
# Load the CSV files in memory
countries = pd.read_csv(countriesURL, sep=';', index_col='Alpha-2 code', keep_default_na=False, na_values=['_'])
countries.info()

<class 'pandas.core.frame.DataFrame'>
Index: 246 entries, AF to ZW
Data columns (total 4 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   English short name lower case  246 non-null    object
 1   Alpha-3 code                   246 non-null    object
 2   Numeric code                   246 non-null    int64 
 3   ISO 3166-2                     246 non-null    object
dtypes: int64(1), object(3)
memory usage: 9.6+ KB


In [10]:
# Connect to the DB
driver = GraphDatabase.driver(params.uri, auth=(params.dbname, params.dbpsw))
# Create a session
session = driver.session()

In [11]:
%%time 
#measure execution time

# Iterate over the countries dataframe
for index, row in countries.iterrows():
    
    country = str(index)
    name = str(row['English short name lower case'])
    # Check country or create it
    session.run("MERGE (c:Country{isocode: $isocode})", isocode=country)
    # Add node property
    session.run("MATCH (c:Country{isocode: $isocode}) SET c.name = $name", isocode=country, name=name)
    

CPU times: user 380 ms, sys: 37.6 ms, total: 418 ms
Wall time: 13.4 s


In [12]:
driver.close()

### Airports and Cities
In this section we model the airports and the cities. In order to match efficiently each airport with the correct city we create a dictionary of cities indexed with the iso-code of the country each city belongs to. Then, if we find data about new cities in the airports file we add them to our cities modelling. 

#### Indexing the Cities
Firstly, we create the dictionary with the cities indexed by country. 

In [13]:
# Load the CSV files in memory
cities = pd.read_csv(citiesURL, sep=',', index_col='geonameid', keep_default_na=False, na_values=['_'])
cities.info()
countries = pd.read_csv(countriesURL, sep=';', index_col='English short name lower case', keep_default_na=False, na_values=['_'])
countries.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23018 entries, 3040051 to 1106542
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        23018 non-null  object
 1   country     23018 non-null  object
 2   subcountry  23018 non-null  object
dtypes: object(3)
memory usage: 719.3+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 246 entries, Afghanistan to Zimbabwe
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Alpha-2 code  246 non-null    object
 1   Alpha-3 code  246 non-null    object
 2   Numeric code  246 non-null    int64 
 3   ISO 3166-2    246 non-null    object
dtypes: int64(1), object(3)
memory usage: 9.6+ KB


In [14]:
# Creation of the structure
citiesByCountry = {}
    
# Iterate over the cities dataframe
for index, row in cities.iterrows():
    
    ## Match country
    cName = str(row['country'])
    # check if the country exists
    # country.index == x returns an array of booleans, thus we need to use the any() method
    if((countries.index == cName).any() == True):
        #get the country code, convert to string and get the lower case to match the country codes in the contology 
        code = str(countries[countries.index == cName]['Alpha-2 code'][0])
        matched = False
        if(code not in citiesByCountry): #create a new country before appending the city
            citiesByCountry.update({code:[]})
        #check we do not insert again the same city
        else:
            for city in citiesByCountry[code]:
                if(city[0] == str(row['name'])):
                    matched = True
                    break
        if(not matched):
            #append the city to the country   
            citiesByCountry[code].append([row['name'], "C"+str(index)])
        
print("Dictionary Created")


Dictionary Created


#### Airports Modelling

In [15]:
# Load the CSV files in memory
airports = pd.read_csv(airportsURL, sep=',', index_col='ident')
airports.info()

<class 'pandas.core.frame.DataFrame'>
Index: 57421 entries, 00A to ZZZZ
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   type          57421 non-null  object 
 1   name          57421 non-null  object 
 2   elevation_ft  49608 non-null  float64
 3   continent     28978 non-null  object 
 4   iso_country   57175 non-null  object 
 5   iso_region    57421 non-null  object 
 6   municipality  51527 non-null  object 
 7   gps_code      41561 non-null  object 
 8   iata_code     9225 non-null   object 
 9   local_code    30030 non-null  object 
 10  coordinates   57421 non-null  object 
dtypes: float64(1), object(10)
memory usage: 5.3+ MB


In [16]:
# Connect to the DB
driver = GraphDatabase.driver(params.uri, auth=(params.dbname, params.dbpsw))
# Create a session
session = driver.session()

In [17]:
%%time 
#measure execution time

count=0 
noMatch=0

# Iterate over the airports dataframe
for index, row in airports.iterrows():
    
    if(pd.isnull(row['gps_code'])): continue
        
    airport = str(row['gps_code'])
    
    # Extract coordinates
    longitude = float(str(row['coordinates']).split(', ')[0])
    latitude = float(str(row['coordinates']).split(', ')[1])
    
    # check airport or create it
    session.run("MERGE (a:Airport{icao: $icao})", icao=airport)
    # Set data property
    session.run("MATCH (a:Airport{icao: $icao}) SET a.name=$name, a.type=$aptype, " +
                    "a.coordinates=point({latitude: $lat, longitude: $long, crs: 'wgs-84'})", 
                    icao=airport, name=str(row['name']), aptype=str(row['type']), lat=latitude, long=longitude)
    count+=1

    ## Handle cities, note that not all airports have data about city
    # but every airport that have city data also have iso_country data
    if(not pd.isnull(row['municipality'])):
        
        # Strip space and uppercase to avoid mismatching for multiple names cities
        cName = str(row['municipality']).replace(" ","").lower()
        
        matched = False
        try:
            for city in citiesByCountry[str(row['iso_country'])]:
                
                check = city[0].replace(" ","").lower() 
                
                if(cName == check):
                    
                    city = str(city[1])
                    # Check city or create it
                    session.run("MERGE (c:City{id: $geoid})", geoid=city)
                    # Create the relationship
                    session.run("MATCH (c:City{id: $geoid}),(a:Airport{icao: $icao}) " +
                                   "MERGE (a)-[:LOCATED]->(c)", geoid=city, icao=airport)
                    matched = True
                    # Once we find the correct city there is no need to keep iterating
                    break 
            # No match with existing cities, we create a new city        
            if(not matched): 
                noMatch += 1
                newCode = "N"+str(noMatch).zfill(6)  
                # Add the new city to the dictionary
                citiesByCountry[str(row['iso_country'])].append([str(row['municipality']), newCode])
                # Check city or create it
                session.run("MERGE (c:City{id: $geoid})", geoid=newCode)
                # Create the relationship
                session.run("MATCH (c:City{id: $geoid}),(a:Airport{icao: $icao}) " +
                               "MERGE (a)-[:LOCATED]->(c)", geoid=newCode, icao=airport)    
                
        # We skip airport countries not in the dictionary
        except KeyError: continue
    
    if(count==1000): print("*** Added the first 1K airports ***")
     
    if(count%10000==0): print("*** 10K Airports added ***")
               
print('***')        
print('*** ADDED IN TOTAL ' + str(count) + ' AIRPORTS ***')
print('***') 


*** Added the first 1K airports ***
*** 10K Airports added ***
*** 10K Airports added ***
*** 10K Airports added ***
*** 10K Airports added ***
***
*** ADDED IN TOTAL 41561 AIRPORTS ***
***
CPU times: user 4min 59s, sys: 18.5 s, total: 5min 18s
Wall time: 56min 41s


In [18]:
driver.close()

#### Cities Modelling
We model the cities using the dictionary we already created so we avoid matching again the countries and cities files.

In [19]:
# Connect to the DB
driver = GraphDatabase.driver(params.uri, auth=(params.dbname, params.dbpsw))
# Create a session
session = driver.session()

In [20]:
%%time 
#measure execution time

count=0
# Iterate over the dictionary
for country in citiesByCountry:
    # Check country or create it
    session.run("MERGE (c:Country{isocode: $isocode})", isocode=country)
    for city in citiesByCountry[country]:
        
        code = str(city[1])
        # Check city or create it
        session.run("MERGE (c:City{id: $geoid})", geoid=code)
        # Add node property
        session.run("MATCH (c:City{id: $geoid}) SET c.name = $name", 
                        geoid=code, name=str(city[0]))
        # Create the relationship
        session.run("MATCH (ct:City{id: $geoid}),(cn:Country{isocode: $isocode}) " +
                               "MERGE (ct)-[:BELONGSTO]->(cn)", geoid=code, isocode=country) 
        
        count+=1
        if(count==1000): print("*** Added the first 1K cities ***")
        if(count%10000==0): print("*** 10K Cities added ***")

print("Modelled " + str(count) + " cities")   

*** Added the first 1K cities ***
*** 10K Cities added ***
*** 10K Cities added ***
*** 10K Cities added ***
Modelled 35352 cities
CPU times: user 2min 5s, sys: 11.9 s, total: 2min 17s
Wall time: 35min 10s


In [21]:
driver.close()

### Manufacturers

In this section we ingest the manufacturers.

In [22]:
# Connect to the DB
driver = GraphDatabase.driver(params.uri, auth=(params.dbname, params.dbpsw))
# Create a session
session = driver.session()

In [23]:
# Load the CSV files in memory
manufacturers = pd.read_csv(manufacturersURL, sep=',', index_col='Code').dropna()
manufacturers.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1547 entries, (any manufacturer) to ZLIN AVIATION
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    1547 non-null   object
dtypes: object(1)
memory usage: 24.2+ KB


In [24]:
# Load the country codes
# We need to convert NaN values to something else otherwise NA strings are converted to NaN -> problem with Namibia
countries = pd.read_csv(countriesURL, sep=';', index_col='English short name lower case', keep_default_na=False, na_values=['_'])

In [25]:
%%time
#measure execution time

# Iterate over the manifacturers dataframe
for index, row in manufacturers.iterrows():
    
    if index == "(any manufacturer)" or row['Name'].startswith("see"):
        continue
        
    man_code = index.replace(" ","").replace("(","").replace(")","")
    
    # Handle the name of the manufacturer 
    splits = row['Name'].split("(")
    man_name = splits[0].strip()
    # Check manufacturer or create it
    session.run("MERGE (m:Manufacturer{code: $code})", code=man_code)
    # Add node property
    session.run("MATCH (m:Manufacturer{code: $code}) SET m.name=$name", 
                                        code=man_code, name = man_name)
    
    # Handle country, find the alpha-2 code of the country
    country = splits[1].replace(")","").strip()
    for ind,cntr in countries.iterrows():
        if ind.lower() == country.lower():
            cn = cntr['Alpha-2 code']
            # Check country or create it
            session.run("MERGE (c:Country{isocode: $isocode})", isocode=cn)
            #add relation
            session.run("MATCH (m:Manufacturer {code: $code}),(c:Country {isocode: $iso}) " +
                        "MERGE (m)-[:HASNATIONALITY]->(c)", code=man_code, iso=cn)

CPU times: user 43.9 s, sys: 1.02 s, total: 44.9 s
Wall time: 2min 28s


In [26]:
driver.close()

### Aircraft Models

In this section we ingest the aircraft models. We consider two file:
- the first contains data about the name and the numberOfSeats;
- the second contains information about engine and manufacturer code

In [27]:
# Connect to the DB
driver = GraphDatabase.driver(params.uri, auth=(params.dbname, params.dbpsw))
# Create a session
session = driver.session()

In [28]:
# Load the CSV files in memory
ac_model = pd.read_csv(aircraftURL, sep=';',header=None, index_col=1).dropna()
ac_model.info()

<class 'pandas.core.frame.DataFrame'>
Index: 218 entries, AT43 to T154
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       218 non-null    object
 1   2       218 non-null    object
 2   3       218 non-null    object
 3   4       218 non-null    object
dtypes: object(4)
memory usage: 8.5+ KB


In [29]:
%%time 
#measure execution time

# Iterate over the aircraft-model dataframe
for index, row in ac_model.iterrows():
    if index == "\\N" or index == "N/A" :
        continue
        
    ac_icao = index
    ac_name = row[0]
    
    # Add numberOfSeats property if not null 
    if not (row[3] == "\\N" or row[3] == "N/A" ):
        n_seats = int(row[3])
        
        # Check Aircraft Model or create it
        session.run("MERGE (a:AircraftModel{icao: $icao})", icao=ac_icao)
        # Add Node property
        session.run("MATCH (a:AircraftModel{icao: $icao}) SET a.name=$name, a.numberOfSeats=$seats",
                           icao=ac_icao, name = ac_name, seats = n_seats)
    else:
        # Check Aircraft Model or create it
        session.run("MERGE (a:AircraftModel{icao: $icao})", icao=ac_icao)
        # Add Node property
        session.run("MATCH (a:AircraftModel{icao: $icao}) SET a.name=$name",
                           icao=ac_icao, name = ac_name)

CPU times: user 519 ms, sys: 43.9 ms, total: 562 ms
Wall time: 8.13 s


In [30]:
# Load the CSV files in memory
ac_type = pd.read_csv(aircraftTypesURL, sep=',', index_col='Designator').dropna()
ac_type.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10020 entries, J328 to SAVG
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   AircraftDescription  10020 non-null  object
 1   Description          10020 non-null  object
 2   EngineCount          10020 non-null  object
 3   EngineType           10020 non-null  object
 4   ManufacturerCode     10020 non-null  object
 5   ModelFullName        10020 non-null  object
 6   WTC                  10020 non-null  object
dtypes: object(7)
memory usage: 626.2+ KB


In [31]:
%%time 
#measure execution time

# Iterate over the aircraft-model dataframe
for index, row in ac_type.iterrows():
    if index == "\\N" or index == "N/A" :
        continue

    ac_icao = str(index)
    engine_type = str(row['EngineType'])
    # Check Aircraft Model or create it
    session.run("MERGE (n:AircraftModel{icao: $icao})", icao=ac_icao)
    # Add engine type
    session.run("MATCH (n:AircraftModel{icao: $icao}) SET n.engineType=$enType", icao=ac_icao, enType=engine_type)
    
    # Add the property only if EngineCount is a number
    if(row['EngineCount'] != "C"): 
        engine_count = int(row['EngineCount'])
        session.run("MATCH (n:AircraftModel {icao: $icao}) SET n.engineCount=$enCount", icao=ac_icao, enCount=engine_count)
    
    # Find the manufacturer
    man = row['ManufacturerCode'].replace(" ","").replace("(","").replace(")","")
    #check manufacturer or create it
    session.run("MERGE (m:Manufacturer {code: $code_m})",code_m=man)
    # Man is the code of the manufacturer
    session.run("MATCH (a:AircraftModel {icao: $icao_a}),(m:Manufacturer {code: $code_m}) " +
                       "MERGE (a)-[:BUILTBY]->(m)", icao_a=ac_icao, code_m=man)
    

CPU times: user 50.9 s, sys: 5.09 s, total: 55.9 s
Wall time: 6min 36s


In [32]:
driver.close()

### Airline Companies

In this section we ingest the airline companies.

In [33]:
# Connect to the DB
driver = GraphDatabase.driver(params.uri, auth=(params.dbname, params.dbpsw))
# Create a session
session = driver.session()

In [34]:
# Load the CSV files in memory, we discard rows with no icao code
airline = pd.read_csv(airlineURL, sep=',', index_col='ICAO')
airline.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6281 entries, nan to ORZ
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   # IATA          1435 non-null   object
 1   Airline         6281 non-null   object
 2   Call sign       5745 non-null   object
 3   Country/Region  6270 non-null   object
dtypes: object(4)
memory usage: 245.4+ KB


In [35]:
%%time 
#measure execution time

checklist = []

# Iterate over the airline dataframe
for index, row in airline.iterrows():
    if index == "nan" or str(index).strip() == "":
        continue
    if index not in checklist:
        checklist.append(index)
        # ICAO code of the airline company
        ac_icao = str(index).strip()
        
        # Add Airline name if available
        if(not pd.isnull(row['Airline'])):
            ac_name = row['Airline']
            # Check airline or create it
            session.run("MERGE (a:Airline {icao: $icao})", icao=ac_icao)
            # Add node property
            session.run("MATCH (a:Airline {icao: $icao}) SET a.name=$name", icao=ac_icao, name=ac_name)
        
        # Handle countries, find the alpha-2 code of the country (if data is available)
        if(not pd.isnull(row['Country/Region'])):
            country = row['Country/Region']
            for ind,cntr in countries.iterrows():
                if ind.lower() == country.lower():
                    cn = cntr['Alpha-2 code']
                    # Check country or create it
                    session.run("MERGE (c:Country{isocode: $isocode})", isocode=cn)
                    #add relation
                    session.run("MATCH (a:Airline {icao: $icao}),(c:Country {isocode: $iso}) " +
                               "MERGE (a)-[:HASNATIONALITY]->(c)", icao=ac_icao, iso=cn)
                    

CPU times: user 2min 48s, sys: 3.98 s, total: 2min 52s
Wall time: 9min 21s


In [36]:
driver.close()

### Flights

We use a sample file of 100K flights.

In [37]:
# Connect to the DB
driver = GraphDatabase.driver(params.uri, auth=(params.dbname, params.dbpsw))
# Create a session
session = driver.session()

In [38]:
# Load the CSV files in memory
flights = pd.read_csv(sampledFlightsURL, sep=',', index_col='ECTRL ID')
flights.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 227743250 to 227849442
Data columns (total 17 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   ADEP                        100000 non-null  object 
 1   ADEP Latitude               99846 non-null   float64
 2   ADEP Longitude              99846 non-null   float64
 3   ADES                        100000 non-null  object 
 4   ADES Latitude               99844 non-null   float64
 5   ADES Longitude              99844 non-null   float64
 6   FILED OFF BLOCK TIME        100000 non-null  object 
 7   FILED ARRIVAL TIME          100000 non-null  object 
 8   ACTUAL OFF BLOCK TIME       100000 non-null  object 
 9   ACTUAL ARRIVAL TIME         100000 non-null  object 
 10  AC Type                     100000 non-null  object 
 11  AC Operator                 100000 non-null  object 
 12  AC Registration             99638 non-null   object 
 13  ICA

In [39]:
%%time 
#measure execution time

count = 0

# Iterate over the flights dataframe
for index, row in flights.iterrows():
        
    count += 1
    # Create the flight
    session.run("MERGE (f:Flight{id: $ectrl})", ectrl=str(index))
    
    # Extracting the properties
    mkt = str(row['STATFOR Market Segment'])
    dist = int(row['Actual Distance Flown (nm)'])
    # Add node properties
    session.run("MATCH (f:Flight{id: $ectrl}) SET f.marketSegment=$mkt, f.distance=$dist", 
                       ectrl=str(index), mkt=mkt, dist=dist)
    
    # Put timestamps in the right dateTime format
    fob = datetime.strptime(str(row['FILED OFF BLOCK TIME']),'%d-%m-%Y %H:%M:%S',).strftime('%Y-%m-%dT%H:%M:%S')
    far = datetime.strptime(str(row['FILED ARRIVAL TIME']),'%d-%m-%Y %H:%M:%S',).strftime('%Y-%m-%dT%H:%M:%S')
    aob = datetime.strptime(str(row['ACTUAL OFF BLOCK TIME']),'%d-%m-%Y %H:%M:%S',).strftime('%Y-%m-%dT%H:%M:%S')
    arr = datetime.strptime(str(row['ACTUAL ARRIVAL TIME']),'%d-%m-%Y %H:%M:%S',).strftime('%Y-%m-%dT%H:%M:%S')
    
    # Handle airports, discard unknown airports
    if(not (str(row['ADEP'])=='ZZZZ')):
        adep = str(row['ADEP'])
        # check airport or create it
        session.run("MERGE (a:Airport{icao: $icao})", icao=adep)
        # create the relationship
        session.run("MATCH (a:Airport{icao: $icao}),(f:Flight{id: $ectrl}) " +
                   "MERGE (f)-[:DEPARTS]->(a)", icao=adep, ectrl=str(index))
        # Add relationship property 
        session.run("MATCH (:Flight{id: $ectrl})-[d:DEPARTS]->(:Airport{icao: $icao}) " +
                   "SET d.filedOffBlock=datetime($fob), d.actualOffBlock=datetime($aob)",
                        icao=adep, ectrl=str(index), fob=fob, aob=aob)
    # Same for destination    
    if(not (str(row['ADES'])=='ZZZZ')):
        ades = str(row['ADES'])
        session.run("MERGE (a:Airport{icao: $icao})", icao=ades)
        # create the relationship
        session.run("MATCH (a:Airport{icao: $icao}),(f:Flight{id: $ectrl}) " +
                   "MERGE (f)-[:ARRIVES]->(a)", icao=ades, ectrl=str(index))
        # Add relationship property 
        session.run("MATCH (:Flight{id: $ectrl})-[a:ARRIVES]->(:Airport{icao: $icao}) " +
                   "SET a.filedArrival=datetime($far), a.actualArrival=datetime($arr)",
                        icao=ades, ectrl=str(index), far=far, arr=arr)
        
    ## Handle airline, discard unknown operator
    if(not (str(row['AC Operator'])=='ZZZ')):
        airline = str(row['AC Operator'])
        # Check airline or create it
        session.run("MERGE (a:Airline{icao: $icao})", icao=airline)
        # Create the relationship
        session.run("MATCH (a:Airline{icao: $icao}),(f:Flight{id: $ectrl}) " +
                    "MERGE (f)-[:PERFORMEDBY]->(a)", icao=airline, ectrl=str(index))
           
    # Handle aircraft models, discard unknwon data
    if(not (str(row['AC Type'])=='ZZZZ')):
        model = str(row['AC Type'])
        # Check aircraft model or create it
        session.run("MERGE (a:AircraftModel{icao: $icao})", icao=model)
        # Create the relationship
        session.run("MATCH (a:AircraftModel{icao: $icao}),(f:Flight{id: $ectrl}) " +
                   "MERGE (a)-[:USEDBY]->(f)", icao=model, ectrl=str(index))
        # Add relationship property about the specific aircraft, if data is available   
        if(not pd.isnull(row['AC Registration'])):
            ac = str(row['AC Registration'])
            session.run("MATCH (a:AircraftModel{icao: $icao})-[p:USEDBY]->(f:Flight{id: $ectrl}) " +
                            "SET p.registration = $ac", icao=model, ectrl=str(index), ac=ac)
                
    # Diagnostics to keep track of progress
    if(count == 1000): print("added the first 1K flights")
 
    if(count%10000 == 0): print("*** Added 10K flights ***")

        
print('***')        
print('*** ADDED IN TOTAL ' + str(count) + ' FLIGHTS ***')
print('***')


added the first 1K flights
*** Added 10K flights ***
*** Added 10K flights ***
*** Added 10K flights ***
*** Added 10K flights ***
*** Added 10K flights ***
*** Added 10K flights ***
*** Added 10K flights ***
*** Added 10K flights ***
*** Added 10K flights ***
*** Added 10K flights ***
***
*** ADDED IN TOTAL 100000 FLIGHTS ***
***
CPU times: user 24min 17s, sys: 2min 11s, total: 26min 28s
Wall time: 6h 7min 32s


In [40]:
driver.close()

### Flight Information Regions (FIRs)

#### Flights-FIRs

We use a sample of 100K flights from the file containing March flights in order to reduce the computational time. For this reason, we use a sampled file for FIRs as well.

In [41]:
# Connect to the DB
driver = GraphDatabase.driver(params.uri, auth=(params.dbname, params.dbpsw))
# Create a session
session = driver.session()

In [42]:
# Load the CSV files in memory
firs = pd.read_csv(sampledFIRsURL, sep=',')
firs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 773544 entries, 0 to 773543
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   ECTRL ID         773544 non-null  int64 
 1   Sequence Number  773544 non-null  int64 
 2   FIR ID           773544 non-null  object
 3   Entry Time       773544 non-null  object
 4   Exit Time        773544 non-null  object
dtypes: int64(2), object(3)
memory usage: 29.5+ MB


In [43]:
%%time 
#measure execution time

# Iterate over the FIRs dataframe
for index, row in firs.iterrows():
        
    # Extracting the properties
    flight = str(row['ECTRL ID'])
    fir = str(row['FIR ID'])
    seq = int(row['Sequence Number'])
    
    #find the type of the airspace
    air_type = "FIR"
    if fir.endswith("FIR"):
        fir = fir.replace("FIR","")
    if fir.endswith("UIR"):
        air_type = "UIR"
        fir = fir.replace("UIR","")
    
    # Put timestamps in the right dateTime format
    intime = datetime.strptime(str(row['Entry Time']),'%d-%m-%Y %H:%M:%S',).strftime('%Y-%m-%dT%H:%M:%S')
    outtime = datetime.strptime(str(row['Exit Time']),'%d-%m-%Y %H:%M:%S',).strftime('%Y-%m-%dT%H:%M:%S')
    
    # Check flight or create it
    session.run("MERGE (f:Flight{id: $ectrl})", ectrl=flight)
    # Check FIR or create it
    session.run("MERGE (f:Airspace{icao: $icao, type: $ty})", icao=fir, ty=air_type)
    # Create the relationship
    session.run("MATCH (fr:Airspace{icao: $icao, type: $ty}),(fl:Flight{id: $ectrl}) " +
                "MERGE (fl)-[:TRAVERS]->(fr)", icao=fir, ty=air_type, ectrl=flight)
    # Add relationship properties
    session.run("MATCH (:Flight{id: $ectrl})-[t:TRAVERS]->(:Airspace{icao: $icao, type: $ty})" +
                "SET t.sequence=$seq, t.entryTime=datetime($entry), t.exitTime=datetime($out)", 
                icao=fir, ty=air_type, ectrl=flight, seq=seq, entry=intime, out=outtime)
    

    #Diagnostics to keep track of progress
    if(index+1 == 1000): print("added the first 1K checkpoints")
             
    if((index+1)%100000==0): print("*** 100k Checkpoints Added ***")
                                  
        
print('***')        
print('*** ADDED IN TOTAL ' + str(index+1) + ' CHECKPOINTS ***')
print('***')


added the first 1K checkpoints
*** 100k Checkpoints Added ***
*** 100k Checkpoints Added ***
*** 100k Checkpoints Added ***
*** 100k Checkpoints Added ***
*** 100k Checkpoints Added ***
*** 100k Checkpoints Added ***
*** 100k Checkpoints Added ***
***
*** ADDED IN TOTAL 773544 CHECKPOINTS ***
***
CPU times: user 43min 15s, sys: 4min 10s, total: 47min 25s
Wall time: 13h 2min 25s


In [44]:
driver.close()

### FIRs coordinates

In [45]:
# Connect to the DB
driver = GraphDatabase.driver(params.uri, auth=(params.dbname, params.dbpsw))
# Create a session
session = driver.session()

In [46]:
# Load the CSV files in memory
firCoord = pd.read_csv(firCoordURL, sep=',',  keep_default_na=False, na_values=['_'])
firCoord.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20086 entries, 0 to 20085
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Airspace ID       20086 non-null  object 
 1   Min Flight Level  20086 non-null  int64  
 2   Max Flight Level  20086 non-null  int64  
 3   Sequence Number   20086 non-null  int64  
 4   Latitude          20086 non-null  float64
 5   Longitude         20086 non-null  float64
dtypes: float64(2), int64(3), object(1)
memory usage: 941.7+ KB


In [47]:
%%time 
#measure execution time

# Iterate over the fir coordinates dataframe
dictionary = {}
for index in range(0,len(firCoord)):
    
    #get every information needed
    air_id = firCoord['Airspace ID'][index]
    seq = int(firCoord['Sequence Number'][index])
    latitude = firCoord['Latitude'][index]
    longitude = firCoord['Longitude'][index]
    
    if air_id not in dictionary:
        dictionary[air_id] = seq
    elif seq == dictionary[air_id]:
        continue
    else:
        dictionary[air_id] = seq
        
    #find the type of the airspace
    air_type = "FIR"
    if air_id.endswith("FIR"):
        air_id = air_id.replace("FIR","")
    if air_id.endswith("UIR"):
        air_type = "UIR"
        air_id = air_id.replace("UIR","")
        
    
    # check fir or create it
    session.run("MERGE (f:Airspace{icao: $icao, type:$ty})", icao=air_id, ty=air_type)
    # Set relation with data property
    session.run("MATCH (f:Airspace{icao: $icao, type:$ty}) " +
                "MERGE (f)-[:BOUNDARYPOINT{sequence: $sequence, " +
                "coordinates: point({latitude: $lat, longitude: $long, crs: 'wgs-84'})}]->(f)", 
                icao=air_id, ty=air_type, sequence=seq, lat=latitude, long=longitude)


CPU times: user 35.3 s, sys: 3.37 s, total: 38.6 s
Wall time: 8min 51s


In [48]:
driver.close()

#### FIRs data from Wikipedia

Here we ingest the information about FIRs contained in the file scraped from Wikipedia.

In [7]:
# Connect to the DB
driver = GraphDatabase.driver(params.uri, auth=(params.dbname, params.dbpsw))
# Create a session
session = driver.session()

In [8]:
# Load the CSV files in memory
fir_wiki = pd.read_csv(firWikiUrl, sep=',', index_col=0).dropna()
fir_wiki.info()
countries = pd.read_csv(countriesURL, sep=';', index_col='English short name lower case', keep_default_na=False, na_values=['_'])
countries.info()

<class 'pandas.core.frame.DataFrame'>
Index: 306 entries, AGGG to ZYSH
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Type      306 non-null    object
 1   ACC_name  306 non-null    object
 2   country   306 non-null    object
dtypes: object(3)
memory usage: 9.6+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 246 entries, Afghanistan to Zimbabwe
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Alpha-2 code  246 non-null    object
 1   Alpha-3 code  246 non-null    object
 2   Numeric code  246 non-null    int64 
 3   ISO 3166-2    246 non-null    object
dtypes: int64(1), object(3)
memory usage: 9.6+ KB


In [9]:
%%time 
#measure execution time

# Iterate over the fir_wiki dataframe
for index, row in fir_wiki.iterrows():
    
    acc_name = row['ACC_name']
    air_type = row['Type']
    
    # Create the FIR 
    session.run("MERGE (f:Airspace {icao: $icao, type: $ty})", icao=index, ty=air_type)
    # Add node property
    session.run("MATCH (f:Airspace {icao: $icao, type: $ty}) SET f.name=$name", icao=index, ty=air_type, name=acc_name)

    # Handle countries, find the alpha-2 code of the country
    country = row['country']
    for ind,cntr in countries.iterrows():
        if ind.lower() == country.lower():
            cn = cntr['Alpha-2 code']
            # Check country or create it
            session.run("MERGE (c:Country{isocode: $iso})", iso=cn)
            session.run("MATCH (f:Airspace {icao: $icao, type: $ty}),(c:Country {isocode: $iso}) " +
                       "MERGE (f)-[:BELONGSTO]->(c)", icao=index, ty=air_type, iso=cn)
    #try to add a name to a UIR if exists an airspace that is a FIR with the same ICAO code
    if air_type == "FIR":
        session.run("MATCH (f:Airspace {icao: $icao, type: $ty}) SET f.name=$name", icao=index, ty="UIR", name=acc_name)

        # Handle countries, find the alpha-2 code of the country
        country = row['country']
        for ind,cntr in countries.iterrows():
            if ind.lower() == country.lower():
                cn = cntr['Alpha-2 code']
                # Check country or create it
                session.run("MERGE (c:Country{isocode: $iso})", iso=cn)
                session.run("MATCH (f:Airspace {icao: $icao, type: $ty}),(c:Country {isocode: $iso}) " +
                           "MERGE (f)-[:BELONGSTO]->(c)", icao=index, ty="UIR", iso=cn)

CPU times: user 4.61 s, sys: 86.4 ms, total: 4.69 s
Wall time: 9.69 s


In [10]:
driver.close()